In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 12)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=4096,
    buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
model.set_parameters("trained_models/g1-reptile/sac")

In [5]:
model.learn(
    total_timesteps=int(1e8), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

Logging to ./logs/sac/reptile_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 81.7     |
|    ep_rew_mean     | -435     |
| time/              |          |
|    episodes        | 100      |
|    fps             | 179      |
|    time_elapsed    | 53       |
|    total_timesteps | 9504     |
| train/             |          |
|    actor_loss      | -377     |
|    critic_loss     | 102      |
|    ent_coef        | 0.853    |
|    ent_coef_loss   | -1.82    |
|    learning_rate   | 0.0001   |
|    n_updates       | 783      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 70.7     |
|    ep_rew_mean     | -136     |
| time/              |          |
|    episodes        | 200      |
|    fps             | 176      |
|    time_elapsed    | 107      |
|    total_timesteps | 18912    |
| train/             |          |
|    actor_loss      | -358     |
|    critic_loss

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 713       |
|    ep_rew_mean     | 5.54e+03  |
| time/              |           |
|    episodes        | 1600      |
|    fps             | 172       |
|    time_elapsed    | 2975      |
|    total_timesteps | 512004    |
| train/             |           |
|    actor_loss      | -3.84e+03 |
|    critic_loss     | 2.44e+04  |
|    ent_coef        | 0.798     |
|    ent_coef_loss   | 0.137     |
|    learning_rate   | 0.0001    |
|    n_updates       | 42658     |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 540      |
|    ep_rew_mean     | 4.45e+03 |
| time/              |          |
|    episodes        | 1700     |
|    fps             | 172      |
|    time_elapsed    | 3182     |
|    total_timesteps | 547512   |
| train/             |          |
|    actor_loss      | -4.5e+03 |
|    critic_loss     | 3.88e+04

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 201      |
|    ep_rew_mean     | -642     |
| time/              |          |
|    episodes        | 3000     |
|    fps             | 171      |
|    time_elapsed    | 4901     |
|    total_timesteps | 838608   |
| train/             |          |
|    actor_loss      | -9.9e+03 |
|    critic_loss     | 3.39e+05 |
|    ent_coef        | 1.79     |
|    ent_coef_loss   | 0.0667   |
|    learning_rate   | 0.0001   |
|    n_updates       | 69875    |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 185       |
|    ep_rew_mean     | -1.26e+03 |
| time/              |           |
|    episodes        | 3100      |
|    fps             | 171       |
|    time_elapsed    | 5020      |
|    total_timesteps | 858780    |
| train/             |           |
|    actor_loss      | -9.61e+03 |
|    critic_loss     | 4.25e+05  |
| 

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 183       |
|    ep_rew_mean     | -1.39e+03 |
| time/              |           |
|    episodes        | 4400      |
|    fps             | 170       |
|    time_elapsed    | 6157      |
|    total_timesteps | 1050624   |
| train/             |           |
|    actor_loss      | -1.09e+04 |
|    critic_loss     | 4.65e+05  |
|    ent_coef        | 2.03      |
|    ent_coef_loss   | -0.0102   |
|    learning_rate   | 0.0001    |
|    n_updates       | 87543     |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 143       |
|    ep_rew_mean     | -1.4e+03  |
| time/              |           |
|    episodes        | 4500      |
|    fps             | 170       |
|    time_elapsed    | 6245      |
|    total_timesteps | 1065516   |
| train/             |           |
|    actor_loss      | -1.02e+04 |
|    critic_loss    

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 123       |
|    ep_rew_mean     | -1.35e+03 |
| time/              |           |
|    episodes        | 5800      |
|    fps             | 170       |
|    time_elapsed    | 7327      |
|    total_timesteps | 1248228   |
| train/             |           |
|    actor_loss      | -9.94e+03 |
|    critic_loss     | 6.23e+05  |
|    ent_coef        | 2.44      |
|    ent_coef_loss   | 0.0836    |
|    learning_rate   | 0.0001    |
|    n_updates       | 104010    |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 174       |
|    ep_rew_mean     | -1.65e+03 |
| time/              |           |
|    episodes        | 5900      |
|    fps             | 170       |
|    time_elapsed    | 7424      |
|    total_timesteps | 1264560   |
| train/             |           |
|    actor_loss      | -9.46e+03 |
|    critic_loss    

...

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 239       |
|    ep_rew_mean     | -168      |
| time/              |           |
|    episodes        | 10000     |
|    fps             | 169       |
|    time_elapsed    | 14641     |
|    total_timesteps | 2478900   |
| train/             |           |
|    actor_loss      | -4.23e+03 |
|    critic_loss     | 1.79e+05  |
|    ent_coef        | 1.04      |
|    ent_coef_loss   | -0.0178   |
|    learning_rate   | 0.0001    |
|    n_updates       | 206566    |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 224       |
|    ep_rew_mean     | -1.25e+03 |
| time/              |           |
|    episodes        | 10100     |
|    fps             | 169       |
|    time_elapsed    | 14792     |
|    total_timesteps | 2504268   |
| train/             |           |
|    actor_loss      | -3.8e+03  |
|    critic_loss    

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 235       |
|    ep_rew_mean     | -2.11e+03 |
| time/              |           |
|    episodes        | 11400     |
|    fps             | 169       |
|    time_elapsed    | 16516     |
|    total_timesteps | 2793828   |
| train/             |           |
|    actor_loss      | -1.84e+03 |
|    critic_loss     | 7.59e+04  |
|    ent_coef        | 0.76      |
|    ent_coef_loss   | -0.165    |
|    learning_rate   | 0.0001    |
|    n_updates       | 232810    |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 285       |
|    ep_rew_mean     | -2.78e+03 |
| time/              |           |
|    episodes        | 11500     |
|    fps             | 169       |
|    time_elapsed    | 16667     |
|    total_timesteps | 2819148   |
| train/             |           |
|    actor_loss      | -1.86e+03 |
|    critic_loss    

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 320       |
|    ep_rew_mean     | -424      |
| time/              |           |
|    episodes        | 12800     |
|    fps             | 168       |
|    time_elapsed    | 18795     |
|    total_timesteps | 3176040   |
| train/             |           |
|    actor_loss      | -707      |
|    critic_loss     | 1.38e+04  |
|    ent_coef        | 0.442     |
|    ent_coef_loss   | -7.89e-05 |
|    learning_rate   | 0.0001    |
|    n_updates       | 264661    |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 536      |
|    ep_rew_mean     | 332      |
| time/              |          |
|    episodes        | 12900    |
|    fps             | 168      |
|    time_elapsed    | 19069    |
|    total_timesteps | 3222084  |
| train/             |          |
|    actor_loss      | -603     |
|    critic_loss     | 1.33e+04

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | 9.02e+03 |
| time/              |          |
|    episodes        | 14300    |
|    fps             | 168      |
|    time_elapsed    | 23440    |
|    total_timesteps | 3960168  |
| train/             |          |
|    actor_loss      | -508     |
|    critic_loss     | 6e+03    |
|    ent_coef        | 0.311    |
|    ent_coef_loss   | -0.101   |
|    learning_rate   | 0.0001   |
|    n_updates       | 330005   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.26e+03 |
|    ep_rew_mean     | 1.29e+04 |
| time/              |          |
|    episodes        | 14400    |
|    fps             | 168      |
|    time_elapsed    | 24180    |
|    total_timesteps | 4085232  |
| train/             |          |
|    actor_loss      | -531     |
|    critic_loss     | 6.33e+03 |
|    ent_coef 

KeyboardInterrupt: 

In [6]:
model.save("trained_models/g1-reptile/sac")